* userId : 각 유저의 고유한 아이디
* productId : 각 상품의 고유한 아이디
* Rating : 각 유저가 각 상품에 대해 평가한 평점
* timestamp : 평가 시간 (본 알고리즘에서는 무시함)


In [1]:
from google.colab import drive
import pandas as pd

drive.mount("/content/gdrive")
default_path='/content/gdrive/MyDrive/ESAA 2학기/데이터/'

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Collaborative Filtering 이용

In [2]:
# Surprise 패키지를 이용하기 위해 설치

In [3]:
!pip3 install surprise

In [4]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split

### 1) Data 가져오기

In [ ]:
# new_df = electronics_data.groupby("productId").filter(lambda x : x['Rating'].count() >= 50)
# new_df.drop(['timestamp'], axis = 1, inplace = True)
# dataframe = pd.DataFrame(new_df)
# dataframe.to_csv("c:/temp/new_df4.csv", index = False)

In [5]:
new_df = pd.read_csv(default_path + "new_df4.csv")

In [6]:
new_df.head()

,userId,productId,Rating
0,A1BKC6B7YHYTVV,0972683275,4.0
1,AWVFSIB42LHKJ,0972683275,4.0
2,A36MQBNADRH8YY,0972683275,5.0
3,A3SRXMPLAEZ6DP,0972683275,4.0
4,A20XXTXWF2TCPY,0972683275,5.0


In [7]:
new_df.shape

(5374313, 3)

In [8]:
reader = Reader(rating_scale = (1,5))
data = Dataset.load_from_df(new_df, reader)

### 2) Trainset, Testset으로 나누기

In [9]:
trainset, testset = train_test_split(data, test_size = 0.3, random_state = 10)

### 3) SVD을 이용 trainset에 파라미터 맞추기 (KNN은 RAM 부족으로 실행 불가)

In [11]:
from surprise import SVD

# algo = KNNWithMeans(k=5, sim_options = {'name': 'pearson_baseline', 'user_based':False})   -> KNN을 이용 trainset에 파라미터 맞추기
algo = SVD(n_factors = 5)
algo.fit(trainset)

### 4) Test accuracy 측정

In [12]:
test_pred = algo.test(testset)

In [13]:
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose = True)

Item-based Model : Test Set
RMSE: 1.2590


1.258973402526316

## Model-based collaborative filtering system

In [14]:
# 대량의 상품, 많은 사용자들에게 추천 가능, sparse한 matrices에도 사용가능

In [15]:
new_df1 = new_df.head(10000)
ratings_matrix = new_df1.pivot_table(values ='Rating', index ='userId', columns ='productId',fill_value =0)
ratings_matrix.head()

productId,0972683275,1400501466,1400501520,1400501776,1400532620,1400532655,140053271X,1400532736,1400599997,1400698987,3744295508,6301977173,7214047977,8862935293,9573212919,9575871979,9625993428,9888002198,9966694544,9983891212,9984984354,9985511476,B000001OM4,B000001OM5,B000001OMI,B000001ON6,B00000DM9W,B00000IGBF,B00000J05A,B00000J061,B00000J08Q,B00000J0D2,B00000J0D5,B00000J0D8,B00000J1EJ,B00000J1EP,B00000J1EQ,B00000J1F3,B00000J1GA,B00000J1QK,B00000J1QR,B00000J1SC,B00000J1TX,B00000J1U8,B00000J1UQ,B00000J1V3,B00000J1V5,B00000J3NF,B00000J3Q7,B00000J3UJ,B00000J434,B00000J4EY,B00000J4FS,B00000J4GE,B00000J6WY,B00000JBAT,B00000JBHP,B00000JBPB,B00000JCT8,B00000JCTO,B00000JD34,B00000JDF5,B00000JDF6,B00000JDHV,B00000JFE3,B00000JFIF,B00000JFMK,B00000JHWX,B00000JI4F,B00000JII6,B00000JMUG,B00000JPPI,B00000JSGF,B00000JYLO,B00000JYWQ,B00000K135
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
A01852072Z7B68UHLI5UG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A0266076X6KPZ6CCHGVS,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A0293130VTX2ZXA70JQS,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A030530627MK66BD8V4LN,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
A0571176384K8RBNKGF8O,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [16]:
ratings_matrix.shape

(9832, 76)

In [17]:
X = ratings_matrix.T
X.head()

userId,A01852072Z7B68UHLI5UG,A0266076X6KPZ6CCHGVS,A0293130VTX2ZXA70JQS,A030530627MK66BD8V4LN,A0571176384K8RBNKGF8O,A0590501PZ7HOWJKBGQ4,A0641581307AKT5MAOU0Q,A076219533YHEV2LJO988,A0821988FXKFYX53V4QG,A099626739FNCRNHIKBCG,A1003EYWAQFOUG,A100LLXMXDZHJZ,A10121433FMMZNY18RWZ8,A1013OEQCVS6AR,A1013Q8NLCY56R,A101DJDPGK0Q46,A101IWCCIWQWC9,A10285OSAM1MZK,A102IUFPXVDRB6,A102L0RVI4LHH3,A102RLOGIBBDMW,A102ZABPUPNYCT,A1038957GWRBP375RU5T,A103XY7DJYZHFN,A104VHQ06KFLDT,A1058D8UFZL711,A105AZN9C8UAOJ,A105TOJ6LTVMBG,A106YUCY4SVX1D,A1077JEY0CE3BB,A1079KKA0AXSL3,A107I2HYD2JZ5G,A1089S59XSJT2T,A108LYH0NZ9919,A108UXJTIV8XXY,A10956KAP9F1CO,A109ZR833WIFQS,A10BH1JDT6SNGA,A10BOETDPAFJ4C,A10CRW7XRJBJ2G,...,AZKKZ58QBXZUA,AZKOO6TVQBDAF,AZL95QPWJDM3Y,AZM8PD2KBVXN5,AZMCJ61JS6L3X,AZMY6E8B52L2T,AZN6Z0QRMZYX8,AZNMO6T8G0ZEU,AZNVF1G1GJRR4,AZO3DZAJ8S0D9,AZODG8OG0CETL,AZOFJX627J181,AZOPTWUSS6UM0,AZOTUFIYPVNDT,AZPPZOYTNLEN0,AZPT44204EODM,AZPT6K5K0IVCT,AZPYIAOEAV3OZ,AZQ2YSCWSKWX6,AZQ9DYG38GVBR,AZQJQBJ75DHMS,AZQOWHUHXMIMS,AZQQVI9RHJYH6,AZQTJ6HUS5L2Q,AZRPGTU8A7KD5,AZS8W54PXQ01W,AZSUNAT0BRQDU,AZTC7YC8PKG39,AZTC8ZV20NO1D,AZV9WA9MNT0FB,AZWOPBY75SGAM,AZX0ZDVAFMN78,AZX5LAN9JEAFF,AZX7I110AF0W2,AZXKUK895VGSM,AZXP46IB63PU8,AZYTSU42BZ7TP,AZZGJ2KMWB7R,AZZMV5VT9W7Y8,AZZST8OYL5P4Q
productId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0972683275,0,0,5,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,5,0,4,0,0,0,5,0,0,0,0,0,...,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0
1400501466,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,5,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1400501520,0,0,0,0,0,0,0,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1400501776,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1400532620,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,5,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [18]:
X.shape

(76, 9832)

In [19]:
X.index

Index(['0972683275', '1400501466', '1400501520', '1400501776', '1400532620',
       '1400532655', '140053271X', '1400532736', '1400599997', '1400698987',
       '3744295508', '6301977173', '7214047977', '8862935293', '9573212919',
       '9575871979', '9625993428', '9888002198', '9966694544', '9983891212',
       '9984984354', '9985511476', 'B000001OM4', 'B000001OM5', 'B000001OMI',
       'B000001ON6', 'B00000DM9W', 'B00000IGBF', 'B00000J05A', 'B00000J061',
       'B00000J08Q', 'B00000J0D2', 'B00000J0D5', 'B00000J0D8', 'B00000J1EJ',
       'B00000J1EP', 'B00000J1EQ', 'B00000J1F3', 'B00000J1GA', 'B00000J1QK',
       'B00000J1QR', 'B00000J1SC', 'B00000J1TX', 'B00000J1U8', 'B00000J1UQ',
       'B00000J1V3', 'B00000J1V5', 'B00000J3NF', 'B00000J3Q7', 'B00000J3UJ',
       'B00000J434', 'B00000J4EY', 'B00000J4FS', 'B00000J4GE', 'B00000J6WY',
       'B00000JBAT', 'B00000JBHP', 'B00000JBPB', 'B00000JCT8', 'B00000JCTO',
       'B00000JD34', 'B00000JDF5', 'B00000JDF6', 'B00000JDHV', 'B00000JFE3',

In [20]:
X1 = X

In [21]:
from sklearn.decomposition import TruncatedSVD
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

(76, 10)

In [22]:
decomposed_matrix

array([[ 1.48104508e+02, -1.54621660e-01,  2.45443132e-02,
        -2.30701148e-05, -1.34152012e-01, -4.03908811e-05,
         4.60126954e-02,  9.74211886e-05, -9.32526429e-04,
        -8.45325625e-02],
       [ 6.40649575e-04,  1.28355840e+00,  1.47458337e+00,
         1.88162557e-01,  3.08241558e-03, -8.22846263e-03,
         4.08829934e-02, -1.46113052e-01,  6.13400467e+01,
        -6.88723354e-01],
       [ 4.98496635e-05,  9.65967962e-02,  3.16079655e-01,
         1.15909453e-02,  3.19092812e-03, -3.65511811e-03,
        -1.91465492e-02,  1.36983751e-01,  1.57034869e+00,
        -2.64504989e-01],
       [ 6.51145689e-04,  7.60653476e-01,  5.11238053e-01,
         6.12594118e-02, -9.26590212e-03, -9.72586709e-03,
        -1.56224927e-02,  4.00027623e-01,  4.44097913e+00,
        -6.84192565e-01],
       [ 5.13935640e-04,  7.83281860e-01,  1.09107643e+00,
         3.06940943e-03,  2.92401556e-03, -3.02780985e-02,
        -2.06020282e-01,  6.70183063e-01,  4.74172415e-01,
        -1.

In [24]:
import numpy as np
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

(76, 76)

In [25]:
X.index[75]

'B00000K135'

In [26]:
i = X.index[75]

product_names = list(X.index)
product_ID = product_names.index(i) #index구하기
product_ID

75

In [27]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

(76,)

In [28]:
correlation_product_ID

array([-0.05801502, -0.36815383, -0.5385032 , -0.53537419, -0.84178795,
       -0.04755133, -0.02657623, -0.51723738, -0.45399438, -0.77675104,
        0.57110273, -0.93531461,  0.11444836,  0.81432899,  0.94280611,
       -0.34295082, -0.92221967, -0.07199658,  0.82495215,  0.85511059,
       -0.67768948,  0.9303967 ,  0.98620561,  0.79891058, -0.45975553,
       -0.71194157, -0.02110254,  0.91059972, -0.75049445, -0.44668896,
       -0.85670307,  0.0887661 ,  0.87093376,  0.93684797,  0.48129188,
       -0.87382703,  0.07122228, -0.41597016, -0.13484062, -0.51984009,
        0.23955557,  0.91016703,  0.98081642, -0.98315626, -0.91471438,
        0.66219779, -0.11427113, -0.86736138, -0.66890357, -0.71346094,
        0.92341259, -0.9020755 ,  0.92580077, -0.45816862,  0.95506886,
        0.64752557,  0.96694909, -0.43832114, -0.97689382, -0.98235767,
        0.13709948,  0.15105189, -0.28465505,  0.82502432, -0.86775499,
        0.96104956,  0.76500749, -0.80318602, -0.02306281, -0.38

In [29]:
Recommend = list(X.index[correlation_product_ID > 0.65]) #피어슨 상관계수가 0.65보다 큰 상품들의 index의 list return

Recommend.remove(i) # 이미 구입한 상품은 뺀다

Recommend[0:24] #상위 24개 추천

['8862935293',
 '9573212919',
 '9966694544',
 '9983891212',
 '9985511476',
 'B000001OM4',
 'B000001OM5',
 'B00000IGBF',
 'B00000J0D5',
 'B00000J0D8',
 'B00000J1SC',
 'B00000J1TX',
 'B00000J1V3',
 'B00000J434',
 'B00000J4FS',
 'B00000J6WY',
 'B00000JBHP',
 'B00000JDHV',
 'B00000JFIF',
 'B00000JFMK',
 'B00000JMUG',
 'B00000JYLO']

In [30]:
Recommend

['8862935293',
 '9573212919',
 '9966694544',
 '9983891212',
 '9985511476',
 'B000001OM4',
 'B000001OM5',
 'B00000IGBF',
 'B00000J0D5',
 'B00000J0D8',
 'B00000J1SC',
 'B00000J1TX',
 'B00000J1V3',
 'B00000J434',
 'B00000J4FS',
 'B00000J6WY',
 'B00000JBHP',
 'B00000JDHV',
 'B00000JFIF',
 'B00000JFMK',
 'B00000JMUG',
 'B00000JYLO']